<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza, Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano, Angelo Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Nicolás Acevedo
- Nombre de alumno 2: Fabiola Pizarro


## Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

## Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer material del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

### **Link de repositorio de GitHub:** `https://github.com/nicoacevedor/MDS7202`

# Importamos librerias útiles

In [1]:
from datetime import datetime
import joblib

import numpy as np
import pandas as pd
import optuna
from optuna.samplers import TPESampler
from pandas.api.types import is_datetime64_any_dtype as is_datetime
from sklearn.compose import ColumnTransformer
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer, MinMaxScaler, OneHotEncoder
import xgboost as xgb

C:\Users\Rocky\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [2]:
df = pd.read_csv('sales.csv')
df_types = {
    'id': int,
    'date': 'datetime64[ns]',
    'city': 'category',
    'lat': float,
    'long': float,
    'pop': int,
    'shop': 'category',
    'brand': 'category',
    'container': 'category',
    'capacity': 'category',
    'price': float,
    'quantity': int
}
df = df.astype(df_types)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7456 entries, 0 to 7455
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   id         7456 non-null   int32         
 1   date       7456 non-null   datetime64[ns]
 2   city       7456 non-null   category      
 3   lat        7456 non-null   float64       
 4   long       7456 non-null   float64       
 5   pop        7456 non-null   int32         
 6   shop       7456 non-null   category      
 7   brand      7456 non-null   category      
 8   container  7456 non-null   category      
 9   capacity   7456 non-null   category      
 10  price      7456 non-null   float64       
 11  quantity   7456 non-null   int32         
dtypes: category(5), datetime64[ns](1), float64(3), int32(3)
memory usage: 357.8 KB


C:\Users\Rocky\AppData\Local\Temp\ipykernel_14396\2241364177.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = df.astype(df_types)


## 1.1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [3]:
gen = np.random.default_rng(seed=42)
data = pd.DataFrame(gen.permutation(df), columns=df.columns)
data = data.astype(df_types)
n = data.shape[0]

train_size, val_size = int(0.7*n), int(0.2*n)
train_data = data[:train_size]
validation_data = data[train_size:train_size+val_size]
test_data = data[train_size+val_size:]

In [4]:
def datetime_to_categorical(df: pd.DataFrame):
    df_in = df.copy()
    datetime_column = ''
    for col in df.columns:
        if is_datetime(df[col]):
            datetime_column = col
    if datetime_column == '':
        raise ValueError("Los datos no poseen ninguna columna con formato fecha")

    df_in['day'] = df_in[datetime_column].dt.day
    df_in['day'] = df_in['day'].astype('category')
    df_in['month'] = df_in[datetime_column].dt.month
    df_in['month'] = df_in['month'].astype('category')
    df_in['year'] = df_in[datetime_column].dt.year
    df_in['year'] = df_in['year'].astype('category')
    df_in.drop([datetime_column], axis=1, inplace=True)
    return df_in

In [5]:
numeric_cols = ['id', 'lat', 'long', 'pop', 'price', 'quantity']
categorical_cols = list(set(df.columns) - (set(numeric_cols).union({'date'})))
categorical_cols = list(set(categorical_cols) | set(['day', 'month', 'year']))

transformer = FunctionTransformer(datetime_to_categorical)

scaler = ColumnTransformer([
    ("NumericScaler", MinMaxScaler(), numeric_cols),
    ("CategoricalEncoder", OneHotEncoder(sparse_output=False), categorical_cols)
], remainder="drop", verbose_feature_names_out=False)
scaler.set_output(transform='pandas')

pipeline_dummy = Pipeline([
    ('Transformer', transformer),
    ('Scaler', scaler),
    ('Regressor', DummyRegressor())
])

pipeline_xgb = Pipeline([
    ('Transformer', transformer),
    ('Scaler', scaler),
    ('Regressor', xgb.XGBRegressor())
])


In [6]:
X_train = train_data[list(set(data.columns) - set('quantity'))]
X_validation = validation_data[list(set(data.columns) - set('quantity'))]
X_test = test_data[list(set(data.columns) - set('quantity'))]
y_train = train_data['quantity']
y_validation = validation_data['quantity']
y_test = test_data['quantity']

pipeline_dummy.fit(X=X_train, y=y_train)
joblib.dump(pipeline_dummy, "pipeline_dummy.pkl")
y_pred_dummy = pipeline_dummy.predict(X_validation)

print(f"El error absoluto medio es de {mean_absolute_error(y_validation, y_pred_dummy):.3f}")

El error absoluto medio es de 13439.757


Al ser un valor tan alto, esto nos dice que la predicción no es buena. En el contexto de las ventas, esto indica que las ventas tienen muchos valores distintos, y que no son similares a la media de todas las ventas. Esto tiene sentido ya que la cantidad de ventas depende por lo menos del país, del producto y del precio.

In [7]:
pipeline_xgb.fit(X=X_train, y=y_train)
y_pred_xgb = pipeline_xgb.predict(X_validation)
joblib.dump(pipeline_xgb, "pipeline_xgb.pkl")
print(f"El error absoluto medio es de {mean_absolute_error(y_validation, y_pred_xgb):.3f}")

El error absoluto medio es de 150.760


Este resultado es mucho mejor, ya que el error absoluto medio es un 1% del valor anterior. Esto indica que la predicción hecha por XGBRegressor es mucho más cercana a la realidad que solo tomar la media de los datos.

## 1.2 Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo.

Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?

Nuevamente, guarde su modelo en un archivo .pkl

Nota: Para realizar esta parte, debe apoyarse en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>.

Hint: Para implementar el constraint, se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

In [8]:
pipeline_xgb_constraint = Pipeline([
    ('Transformer', transformer),
    ('Scaler', scaler),
    ('Regressor', xgb.XGBRegressor(monotone_constraints={'price': -1}))
])
pipeline_xgb_constraint.fit(X=X_train, y=y_train)
joblib.dump(pipeline_xgb_constraint, "pipeline_xgb_constraint.pkl")
y_pred_constraint = pipeline_xgb_constraint.predict(X_validation)
print(f"El error absoluto medio es de {mean_absolute_error(y_validation, y_pred_constraint):.3f}")

El error absoluto medio es de 151.987


El error aumenta un poco, no mucho pero aumenta. Esto puede deberse a que anteriormente el modelo ya aprendió la relación inversa entre el precio y el producto, por lo que la variación es poca. Además, si bien existe esta relación inversa, eso no considera otros factores como el precio por país, que aunque este aumente, puede que en proporción el producto sea más barato, por lo que en esa situación la relación ya no es completamente inversa.

Por lo tanto, el amigo no tiene toda la razón.

## 1.3 Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Los hiperparámetros del `XGBRegressor` son los siguientes:
- `learning_rate`: Factor de aprendizaje del modelo
- `n_estimators`: Número de árboles generados
- `max_depth`: Máxima profundidad de cada árbol
- `max_leaves`: Número máximo de hojas de cada árbol
- `min_child_weight`: Umbral para el cual un nodo se deja de dividir si la cantidad de datos baja de este
- `reg_alpha`: Factor de regularización para dimensiones altas
- `reg_lambda`: Factor de regularización para reducir overfitting

El hiperparámetro del `OneHotEncoder`
- `min_frequency`: Mínima frecuencia relativa para una etiqueta ser considerada categoría

In [9]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

random_seed = 42

study = optuna.create_study(
    sampler=TPESampler(seed=random_seed),
    direction='minimize',
    study_name="pipeline"
)

def objective(trial: optuna.Trial, data, target, validation, val_target, random_seed=42):
    regressor_params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 100),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1)
    }
    numeric_cols = ['id', 'lat', 'long', 'pop', 'price', 'quantity']
    categorical_cols = list(set(data.columns) - (set(numeric_cols).union({'date'})))
    categorical_cols = list(set(categorical_cols) | set(['day', 'month', 'year']))

    transformer = FunctionTransformer(datetime_to_categorical)

    min_frequency = trial.suggest_float('min_frequency', 0, 1)
    scaler = ColumnTransformer([
        ("NumericScaler", MinMaxScaler(), numeric_cols),
        ("CategoricalEncoder", OneHotEncoder(sparse_output=False, min_frequency=min_frequency), categorical_cols)
    ], remainder="drop", verbose_feature_names_out=False)
    scaler.set_output(transform='pandas')

    regressor = xgb.XGBRegressor(**regressor_params)

    pipeline = Pipeline([
        ('Transformer', transformer),
        ('Scaler', scaler),
        ('Regressor', regressor), 
    ])

    pipeline.fit(X=data, y=target)
    y_pred = pipeline.predict(X=validation)
    return mean_absolute_error(val_target, y_pred)


study.optimize(
    lambda trial: objective(trial, X_train, y_train, X_validation, y_validation),
    timeout=5*60,
    show_progress_bar=True
)

Best trial: 872. Best value: 126.557:  100%|██████████| 05:00/05:00


In [10]:
min_frequency_opt = study.best_params.pop('min_frequency')

scaler = ColumnTransformer([
    ("NumericScaler", MinMaxScaler(), numeric_cols),
    ("CategoricalEncoder", OneHotEncoder(sparse_output=False, min_frequency=min_frequency_opt), categorical_cols)
], remainder="drop", verbose_feature_names_out=False)
scaler.set_output(transform='pandas')

optimized_pipeline = Pipeline([
    ('Transformer', transformer),
    ('Scaler', scaler),
    ('Regressor', xgb.XGBRegressor(random_seed=42, **study.best_params))
])
joblib.dump(optimized_pipeline, "optimized_pipeline.pkl")

print(f"Número de trials: {len(study.trials)}")
print(f"MAE: {study.best_value:.3f}")
print("\nMejores parámetros:\n--------------------")
for key, value in study.best_params.items():
    print(f"{key}: {value:.3f}")

Número de trials: 1072
MAE: 126.557

Mejores parámetros:
--------------------
learning_rate: 0.082
n_estimators: 97.000
max_depth: 5.000
max_leaves: 25.000
min_child_weight: 1.000
reg_alpha: 0.149
reg_lambda: 0.111
min_frequency: 0.467


El `MAE` es menor al encontrado en la sección anterior, lo que se debe a que se está trabajando con valores mucho más óptimos de los hiperparámetros, en cambio antes se trabajaba con los valores por defecto que, si bien dan un resultado más o menos bueno, no es el mejor. Por lo tanto, al haber reducido el `MAE`, vemos que el nuevo modelo optimizado es mejor que el de la sección anterior.

## 1.4 Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [20]:
optuna.logging.set_verbosity(optuna.logging.WARNING)

random_seed = 42

study = optuna.create_study(
    sampler=TPESampler(seed=random_seed),
    direction='minimize',
    study_name="pipeline_pruning"
)

def objective(trial: optuna.Trial, data, target, validation, val_target, random_seed=42):
    regressor_params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 100),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1)
    }
        
    numeric_cols = ['id', 'lat', 'long', 'pop', 'price', 'quantity']
    categorical_cols = list(set(data.columns) - (set(numeric_cols).union({'date'})))
    categorical_cols = list(set(categorical_cols) | set(['day', 'month', 'year']))
    
    transformer = FunctionTransformer(datetime_to_categorical)

    min_frequency = trial.suggest_float('min_frequency', 0, 1)
    scaler = ColumnTransformer([
        ("NumericScaler", MinMaxScaler(), numeric_cols),
        ("CategoricalEncoder", OneHotEncoder(sparse_output=False, min_frequency=min_frequency), categorical_cols)
    ], remainder="drop", verbose_feature_names_out=False)
    scaler.set_output(transform='pandas')

    pruning_callback = optuna.integration.XGBoostPruningCallback(
        trial, observation_key="validation_0-mae"
    )

    regressor = xgb.XGBRegressor(**regressor_params, eval_metric='mae',
                                early_stopping_rounds=10,
                                callbacks=[pruning_callback])

    # Habilitar la opción enable_categorical=True
    # regressor.set_params(enable_categorical=True)

    pipeline = Pipeline([
        ('Transformer', transformer),
        ('Scaler', scaler),
        ('Regressor', regressor), 
    ])

    # Ajustar el ColumnTransformer con los datos de entrenamiento
    data_transformed = transformer.transform(data)
    scaler.fit(data_transformed)
    # Preprocesamiento de los datos de validación
    validation_transformed = transformer.transform(validation)
    validation_scaled = scaler.transform(validation_transformed)

    pipeline.fit(X=data, y=target, Regressor__eval_set=[(validation_scaled, val_target)])
    y_pred = pipeline.predict(X=validation_scaled)
    return mean_absolute_error(val_target, y_pred)


study.optimize(
    lambda trial: objective(trial, X_train, y_train, X_validation, y_validation),
    timeout=5*60,
    show_progress_bar=True
)

   0%|          | 00:00/05:00

[0]	validation_0-mae:12929.58351
[1]	validation_0-mae:12439.61955
[2]	validation_0-mae:11967.95412
[3]	validation_0-mae:11514.05199
[4]	validation_0-mae:11076.83867
[5]	validation_0-mae:10656.75833
[6]	validation_0-mae:10253.09367
[7]	validation_0-mae:9863.58898
[8]	validation_0-mae:9490.42122
[9]	validation_0-mae:9130.28326
[10]	validation_0-mae:8783.58811
[11]	validation_0-mae:8451.37023
[12]	validation_0-mae:8130.44730
[13]	validation_0-mae:7821.96862
[14]	validation_0-mae:7525.70140
[15]	validation_0-mae:7241.09228
[16]	validation_0-mae:6966.15285
[17]	validation_0-mae:6702.22867
[18]	validation_0-mae:6447.97666
[19]	validation_0-mae:6204.14580
[20]	validation_0-mae:5969.64566
[21]	validation_0-mae:5743.40575
[22]	validation_0-mae:5525.83989
[23]	validation_0-mae:5316.55916
[24]	validation_0-mae:5115.11822
[25]	validation_0-mae:4921.75714
[26]	validation_0-mae:4735.67670
[27]	validation_0-mae:4556.88736
[28]	validation_0-mae:4384.83296
[29]	validation_0-mae:4219.27736
[30]	validati

   0%|          | 00:00/05:00

[W 2023-11-17 10:55:44,531] Trial 0 failed with parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 98, 'max_depth': 8, 'max_leaves': 60, 'min_child_weight': 1, 'reg_alpha': 0.15599452033620265, 'reg_lambda': 0.05808361216819946, 'min_frequency': 0.8661761457749352} because of the following error: ValueError('Los datos no poseen ninguna columna con formato fecha').
Traceback (most recent call last):
  File "C:\Users\Rocky\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\Rocky\AppData\Local\Temp\ipykernel_14396\334774914.py", line 65, in <lambda>
    lambda trial: objective(trial, X_train, y_train, X_validation, y_validation),
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Rocky\AppData\Local\Temp\ipykernel_14396\

ValueError: Los datos no poseen ninguna columna con formato fecha

Prunning permite detener automáticamente las pruebas poco prometedoras en las primeras etapas del entrenamiento de un modelo, en otras palabras, se interrumpe el entrenamiento de un modelo si no muestra un rendimiento prometedor según una métrica específica. Esto debería beneficiar el entrenamiento del modelo porque ayuda a evitar el sobreajuste y mejorar la eficiencia del proceso de búsqueda de hiperparámetros.

## 1.5 Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

- Gráfico de historial de optimización
- Gráfico de coordenadas paralelas
- Gráfico de importancia de hiperparámetros

Comente sus resultados: ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [12]:
# Inserte su código acá

## 1.6 Síntesis de resultados (0.3)

Finalmente, genere una tabla resumen del MAE obtenido en los 5 modelos entrenados (desde Baseline hasta XGBoost con Constraints, Optuna y Prunning) y compare sus resultados. ¿Qué modelo obtiene el mejor rendimiento? 

Por último, cargue el mejor modelo, prediga sobre el conjunto de test y reporte su MAE. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>